<a href="https://colab.research.google.com/github/devangi2000/Deep-Learning/blob/master/FullyConnectedNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports


In [1]:
import torch
import torchvision
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F #includes stuff that don't take params like relu,etc
from torch.utils.data import DataLoader #easier dataset management and creates mini batches
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset

### Create Fully Connected Network

In [2]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__() #calls the initialization method of the parent class (nn.Module here)
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Sanity check
#model = NN(784,10)
#x = torch.randn(64,784)
#print(model(x).shape)

### Set device

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Hyperparameters

In [4]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

### Load Data

In [5]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform = transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform = transforms.ToTensor(), download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

### Initialize Network

In [6]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

### Loss and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train Network

In [8]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in  enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get data to correct shape
        data = data.reshape(data.shape[0], -1) #Flatten

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        #backward
        optimizer.zero_grad() #set all the gradients to zero for each batch so it
        # doesn't store the backprop calculations from previous forward props
        loss.backward() 

        # gradient descent or adam step
        optimizer.step()

In [10]:
def check_accuracy(loader, model):

    if loader.dataset.train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')

    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad(): # unnecessary computation avoidance because we don't need
    # to calculate gradients while checking accuracy
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            # 64x10,
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0) #64
        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()
    
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58347 / 60000 with accuracy 97.25
Checking accuracy on test data
Got 9662 / 10000 with accuracy 96.62
